In [ ]:
## 1) Install & import libraries

# Run this cell in a notebook to install dependencies if they are missing.
# On Windows PowerShell, run: `pip install -r requirements.txt` (see project README)

import re
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [ ]:
class TaglishProcessor:
    def __init__(self, vocab_size=5000, max_length=100, oov_token='<OOV>', num_classes=3):
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.oov_token = oov_token
        self.num_classes = num_classes
        self.tokenizer = None

    @staticmethod
    def _map_label_to_int(label):
        # Accept numeric labels (0/1/2 or 1-5 ratings) or text labels
        if pd.isna(label):
            return 1  # default to Neutral
        try:
            v = int(label)
            # If rating 1-5 map: 4-5 -> Good(2), 3 -> Neutral(1), 1-2 -> Bad(0)
            if v in (0, 1, 2):
                return int(v)
            if 1 <= v <= 5:
                if v >= 4:
                    return 2
                if v == 3:
                    return 1
                return 0
            return int(v)
        except Exception:
            s = str(label).strip().lower()
            if s in ('good', 'positive', 'pos', 'ganda', 'maganda'):
                return 2
            if s in ('neutral', 'neutrality', 'n') or s == 'okay' or s == 'ok':
                return 1
            if s in ('bad', 'negative', 'neg', 'sira', 'hindi maganda'):
                return 0
            # fallback: if phrase contains positive/negative words
            if 'maganda' in s or 'good' in s or 'love' in s or 'ganda' in s:
                return 2
            if 'sira' in s or 'di' in s or 'hindi' in s or 'bad' in s:
                return 0
            return 1

    @staticmethod
    def clean_text(text):
        if pd.isna(text):
            return ""
        # Lowercase
        text = str(text).lower()
        # Remove HTML tags
        text = re.sub(r'<[^>]+>', ' ', text)
        # Normalize whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        # Keep punctuation and stopwords (no removal)
        return text

    def fit_tokenizer(self, texts):
        self.tokenizer = Tokenizer(num_words=self.vocab_size, oov_token=self.oov_token)
        self.tokenizer.fit_on_texts(texts)
        return self.tokenizer

    def texts_to_padded_sequences(self, texts):
        if self.tokenizer is None:
            raise ValueError('Tokenizer not fitted. Call fit_tokenizer first or use load_and_prep_data.')
        sequences = self.tokenizer.texts_to_sequences(texts)
        padded = pad_sequences(sequences, maxlen=self.max_length, padding='post', truncating='post')
        return padded

    def load_and_prep_data(self, filepath, text_col='review_text', label_col='label', test_size=0.15, val_size=0.15, random_state=42):
        df = pd.read_csv(filepath)
        # Clean texts
        df[text_col] = df[text_col].astype(str).map(self.clean_text)

        X = df[text_col].tolist()
        # Map labels to integers 0=Bad,1=Neutral,2=Good
        y_int = df[label_col].map(self._map_label_to_int).astype(int).values

        # Fit tokenizer on full data
        self.fit_tokenizer(X)
        X_padded = self.texts_to_padded_sequences(X)

        # First split off test set
        X_temp, X_test, y_temp, y_test = train_test_split(X_padded, y_int, test_size=test_size, random_state=random_state, stratify=y_int)
        # Now split remaining into train and val. Compute val proportion relative to the temp set.
        val_relative = val_size / (1 - test_size)
        X_train, X_val, y_train_int, y_val_int = train_test_split(X_temp, y_temp, test_size=val_relative, random_state=random_state, stratify=y_temp)

        # One-hot encode labels for training
        y_train_cat = to_categorical(y_train_int, num_classes=self.num_classes)
        y_val_cat = to_categorical(y_val_int, num_classes=self.num_classes)
        y_test_cat = to_categorical(y_test, num_classes=self.num_classes)

        # Return both categorical (for training) and integer labels (for evaluation/prediction)
        return X_train, X_val, X_test, y_train_cat, y_val_cat, y_test_cat, y_train_int, y_val_int, y_test, self.tokenizer


In [ ]:
# Example usage (uncomment and set `path` to your CSV to run):
# processor = TaglishProcessor(vocab_size=5000, max_length=100)
# X_train, X_val, X_test, y_train, y_val, y_test, tokenizer = processor.load_and_prep_data('data/taglish_reviews.csv')
# print('Shapes:', X_train.shape, X_val.shape, X_test.shape)


In [ ]:
# Hugging Face Dataset Loader (Option A)
def load_hf_dataset_to_csv(dataset_id, save_path=r'..\data\hf_dataset.csv', prefer_split='train'):
    """Load a Hugging Face dataset by id and save a CSV with detected text/label columns.
    Replace `dataset_id` with the dataset's repo id from https://huggingface.co/datasets."""
    try:
        # Lazy import; install with `pip install datasets` if missing
        from datasets import load_dataset
    except Exception as e:
        raise RuntimeError('Please install the `datasets` package: pip install datasets') from e
    ds = load_dataset(dataset_id)
    # If dataset has multiple splits, pick the preferred split if available
    if isinstance(ds, dict):
        split = prefer_split if prefer_split in ds else list(ds.keys())[0]
        df = ds[split].to_pandas()
    else:
        df = ds.to_pandas()
    # Heuristic for text column
    text_col_candidates = ['review_text','review','text','content','body']
    label_col_candidates = ['label','labels','sentiment','rating','stars']
    text_col = next((c for c in text_col_candidates if c in df.columns), df.columns[0])
    label_col = next((c for c in label_col_candidates if c in df.columns), None)
    # Save a reduced CSV if label found, otherwise save text-only CSV
    out_df = df[[text_col]] if label_col is None else df[[text_col, label_col]]
    out_path = save_path
    # Ensure parent dir exists
    import os
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    out_df.to_csv(out_path, index=False)
    print(f'Saved dataset to {out_path}\nDetected text column: {text_col}\nDetected label column: {label_col}')
    return out_path, text_col, label_col

# Example (replace the dataset id below with a real HF dataset id):
# load_hf_dataset_to_csv('some-user/electronics-reviews')


In [ ]:
# Export SentiShop sqlite tables to CSV and preview marc3ee datasets
import sqlite3, pandas as pd, os
db_path = 'data/sentishop_db.sqlite3'
if os.path.exists(db_path):
    conn = sqlite3.connect(db_path)
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)['name'].tolist()
    os.makedirs('data/sentishop_tables', exist_ok=True)
    print('Found tables:', tables)
    for t in tables:
        try:
            df = pd.read_sql_query(f"SELECT * FROM '{t}'", conn)
            out = f'data/sentishop_tables/{t}.csv'
            df.to_csv(out, index=False)
            print(f'Exported {t} -> {out} (rows={len(df)})')
        except Exception as e:
            print('Skipping table', t, 'error:', e)
    conn.close()
else:
    print('Database not found at', db_path)

# Preview marc3ee CSVs (first 5 rows) and report shape
for f in ['data/marc3ee_reviews.csv','data/marc3ee_datasetb_withpredictions.csv']:
    if os.path.exists(f):
        df_preview = pd.read_csv(f, nrows=5)
        try:
            total_rows = sum(1 for _ in open(f)) - 1
        except Exception:
            total_rows = 'unknown'
        print(f'\nPreview of {f} (approx rows: {total_rows}):')
        display(df_preview)
    else:
        print('File not found:', f)
